In [51]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [52]:
print('Number of images in the dataset: {}'.format(len(glob.glob('images/*.jpg'))))

Number of images in the dataset: 7390


In [53]:
CATS = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx']
cats_images = []
dogs_images = []

for img in glob.glob('images/*.jpg'):
    if any(cat in img for cat in CATS):
        cats_images.append(img)
    else:
        dogs_images.append(img)
        
print(f"There are {len(cats_images)} cat images")
print(f"There are {len(dogs_images)} dog images")

There are 2400 cat images
There are 4990 dog images


In [54]:
from cProfile import label
from cgi import test


np.random.shuffle(cats_images)
np.random.shuffle(dogs_images)

train_d, val_d, test_d = np.split(dogs_images, [int(len(dogs_images)*0.7), int(len(dogs_images)*0.8)])
train_c, val_c, test_c =np.split(cats_images, [int(len(cats_images)*0.7), int(len(cats_images)*0.8)])


In [55]:
train_dog_df = pd.DataFrame({'image': train_d, 'label': 'dog'})
val_dog_df = pd.DataFrame({'image': val_d, 'label': 'dog'})
test_dog_df = pd.DataFrame({'image': test_d, 'label': 'dog'})

train_cat_df =pd.DataFrame({'image': train_c, 'label': 'cat'})
val_cat_df = pd.DataFrame({'image': val_c, 'label': 'cat'})
test_cat_df = pd.DataFrame({'image': test_c, 'label': 'cat'})

test_dog_df.head()

,image,label
0,images\english_setter_37.jpg,dog
1,images\pug_171.jpg,dog
2,images\scottish_terrier_49.jpg,dog
3,images\leonberger_99.jpg,dog
4,images\saint_bernard_43.jpg,dog


In [56]:
train_df = pd.concat([train_dog_df, train_cat_df])
val_df = pd.concat([val_dog_df, val_cat_df])
test_df = pd.concat([test_dog_df, test_cat_df])

print(f"{len(train_df)} images for training")
print(f"{len(test_df)} images for testing")
print(f"{len(val_df)} images for validation")

5173 images for training
1478 images for testing
739 images for validation


In [57]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

trainGenerator = ImageDataGenerator(rescale=1./225.)
valGenerator = ImageDataGenerator(rescale=1./225.)
testGenerator = ImageDataGenerator(rescale=1./225.)

trainDataset = trainGenerator.flow_from_dataframe(
    dataframe=train_df,
    class_mode="binary",
    x_col="image",
    y_col="label",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH)
)

valDataset = valGenerator.flow_from_dataframe(
    dataframe=val_df,
    class_mode="binary",
    x_col="image",
    y_col="label",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH)
)
    
testDataset = testGenerator.flow_from_dataframe(
  dataframe=test_df,
  class_mode='binary',
  x_col="image",
  y_col="label",
  batch_size=BATCH_SIZE,
  seed=42,
  shuffle=True,
  target_size=(IMG_HEIGHT,IMG_WIDTH)
)

Found 5173 validated image filenames belonging to 2 classes.
Found 739 validated image filenames belonging to 2 classes.
Found 1478 validated image filenames belonging to 2 classes.


In [58]:
images, labels = next(iter(testDataset))

NameError: name 'scipy' is not defined

In [ ]:

print('Batch shape: ', images.shape)
print('Label shape: ', labels.shape)

In [ ]:
plt.imshow(images[3])
print('Label', labels[3])

In [ ]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(256, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(512, (3, 3), activation='relu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 64)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 52, 52, 256)       295168    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 26, 26, 256)     

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs=15
history =model.fit(trainDataset, epochs=epochs, validation_data=(valDataset))

NameError: name 'scipy' is not defined

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'])
plt.show()

In [ ]:
loss, acc = model.evaluate(testDataset)

print('Loss', loss)
print('Accuracy:', acc)